In [1]:
from google.colab import drive
drive.mount('/content/drive/')
!mkdir -p drive 
import os
from os import listdir
import cv2

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
'''
created by nishant sethi
on 1/31/2019
'''

import numpy as np
import time
import tensorflow
from tensorflow.examples.tutorials.mnist import input_data
from keras.initializers import RandomNormal

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt



Using TensorFlow backend.


In [0]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

class DCGAN(object):
    def __init__(self, img_rows=64, img_cols=64, channel=3):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self,d_dropout=0.3):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = d_dropout
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(32, 64, strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(16, 128, strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(8, 256, strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))
        
        self.D.add(Conv2D(4, 512, strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self,g_dropout=0.4,g_train=False):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = g_dropout
        depth = 512
        dim = 4
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100,kernel_initializer=RandomNormal(stddev=0.02)))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(LeakyReLU(alpha=0.2))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
#         self.G.add(UpSampling2D())
#         self.G.add(Conv2DTranspose(512, 5,strides=2, padding='same'))
#         self.G.add(BatchNormalization(momentum=0.9))
#         self.G.add(Activation('relu'))

        #self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(1024, 4,strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(LeakyReLU(alpha=0.2))
        

        self.G.add(Conv2DTranspose(512, 8,strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(LeakyReLU(alpha=0.2))
        
        self.G.add(Conv2DTranspose(256, 16,strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(LeakyReLU(alpha=0.2))
     
        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(3, 64,strides=2, padding='same',kernel_initializer=RandomNormal(stddev=0.02)))
        self.G.add(Activation('tanh'))
        if g_train == True:
          g_learning_rate=0.0002,
          g_beta_1=0.5
          optimizer = Adam(lr=g_learning_rate, beta_1=g_beta_1)
          self.g.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        self.G.summary()
        return self.G

    def discriminator_model(self,d_learning_rate=0.0002,d_beta_1=0.5,d_dropout=0.3):
        if self.DM:
            return self.DM
        optimizer = Adam(lr=d_learning_rate, beta_1=d_beta_1)
        self.DM = Sequential()
        self.DM.add(self.discriminator(d_dropout))
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self,a_learning_rate=0.0002,a_decay=3e-8):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=a_learning_rate, decay=a_decay)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM


In [0]:
class MY_DCGAN(object):
    def __init__(self,rows,colms,channels,d_learning_rate,d_beta_1,path,a_learning_rate,a_decay,g_dropout,d_dropout,batch_size,g_train):
        self.img_rows = rows
        self.img_cols = colms
        self.channels = channels
        self.d_learning_rate=d_learning_rate
        self.d_beta_1=d_beta_1
        self.a_learning_rate=a_learning_rate
        self.a_decay=a_decay
        self.path=path
        self.g_dropout=g_dropout
        self.d_dropout=d_dropout
        self.batch_size=batch_size
        self.g_train=g_train
        
        
        self.x_train =self.load_dataset(self.path).reshape(-1, self.img_rows,\
        	self.img_cols, 3).astype(np.float32)
        
        
        self.DCGAN = DCGAN(self.img_rows,self.img_cols,self.channels)
        self.discriminator =  self.DCGAN.discriminator_model(self.d_learning_rate,self.d_beta_1,self.d_dropout)
        self.adversarial = self.DCGAN.adversarial_model(self.a_learning_rate,self.a_decay)
        self.generator = self.DCGAN.generator(self.g_dropout,self.g_train)
    
    #load the required dataset. 
    def load_dataset(self,train_path):
        images=os.listdir(train_path)
        X=[]
        image_size=64
        for img in images:
          img=os.path.join(train_path,img)
          #print(img)
          image = cv2.imread(img)
          image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
          image = image.astype(np.float32)
          image = np.multiply(image, 1.0 / 255.0)
          X.append(image)
        X_train=np.array(X)
        #print(X_train.shape)
        print("Data loaded")
        return X_train

    def train(self, train_steps=2000, save_interval=0):
        batch_size=self.batch_size
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size)]
            #print("x_train shape=",self.x_train.shape)
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            #print("image_train shape=",images_train.shape," noise shape=",noise.shape)
            images_fake = self.generator.predict(noise)
            #print("image fake=",images_fake.shape)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=False, samples=noise_input.shape[0],\
                        noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = '/content/drive/My Drive/pictures/result.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "/content/drive/My Drive/pictures/result_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols,3])
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()


In [0]:
#@title Hyper Parameters { output-height: 500, form-width: "30%", display-mode: "both" }
no_of_rows = 64 #@param {type:"integer"}
no_of_cols = 64 #@param {type:"number"}
no_of_channels = 3 #@param ["1", "3"] {type:"raw", allow-input: true}
discriminator_learning_rate = 0
discriminator_dropout = 0.4 #@param {type:"number"}
adverserial_learning_rate = 0
generator_dropout = 0.4 #@param {type:"number"}
generator_train = False #@param {type:"boolean"}



if __name__ == '__main__':
    dcgan = MY_DCGAN(
        rows=no_of_rows,                                          # height of image
        colms=no_of_cols,                                         # width of image
        channels=no_of_channels,                                       # number of channels
        d_learning_rate=0.0002,                           # learning rate for discriminator
        d_beta_1=0.5,                                     # beta value for discriminator
        path='/content/drive/My Drive/images/',           # path of data set
        a_learning_rate=0.002,                            # learning rate for adverserial 
        a_decay=3e-8,                                     # decay rate for optimizer of adverserial
        g_dropout=generator_dropout,                      # dropout for generator
        d_dropout=discriminator_dropout,                  # dropout for discriminator
        batch_size=256,                                   # batch size for training 
        g_train=generator_train                                      # true if you want to train generator else false
    )
    timer = ElapsedTimer()
    dcgan.train(train_steps=400, save_interval=50)
    timer.elapsed_time()
    dcgan.plot_images(fake=True)
    dcgan.plot_images(fake=False, save2file=True)

Data loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        8388640   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 16)          8388624  